In [3]:
import librosa
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
from IPython.display import Audio
import numpy as np
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
from sklearn.metrics import confusion_matrix
import IPython.display as ipd  # To play sound in the notebook
import os # interface with underlying OS that python is running on
import sys

In [4]:
# LOAD IN FILE
main_dir = 'D:/Makaleler/makale19 speech recog/EMOVO/'
#main_dir ='D:/Makaleler/makale19 speech recog/ravdes/ex/'
sub_dir = os.listdir(main_dir)
x, sr = librosa.load(main_dir+sub_dir[1]+'/dis-f1-b1.wav')
# PLAY any AUDIO FILE
#librosa.output.write_wav('MaleNeutral.wav', x, sr)
Audio(data=x, rate=sr)

In [6]:
def extract_feature(file_name, offst=0.5):
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast',offset=offst)
    
    stft = np.abs(librosa.stft(X))
    chroma_cq = librosa.feature.chroma_cqt(y=X, sr=sample_rate)
    cqt=np.mean(chroma_cq,axis=1)
    
    #chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate,n_fft=2048).T,axis=0)
    #chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate,n_mels=128,fmax=8000).T,axis=0)
    #mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,cqt,mel,contrast,tonnetz

def extract_featurev2(file_name):
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast', sr=None)   
    #mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=64).T, axis=0)
    C = np.abs(librosa.cqt(X, sr=sr, fmin=librosa.note_to_hz('C2'), n_bins=60))
    #d=librosa.amplitude_to_db(C)
    d=librosa.power_to_db(C)
    cqt = np.mean(d,axis=1)
    return cqt

In [7]:
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

### a= []
b= []
c= []
d= []
e= []
#from warnings import warn
import warnings
# run block of code and catch warnings
with warnings.catch_warnings():
	# ignore all caught warnings
	warnings.filterwarnings("ignore")
	# execute code that will generate warnings

emotion = []
path = []
gender = []

X4= pd.DataFrame()

n=0
for root, dirs, files in os.walk(main_dir, topdown=False):
    for f in files:
        if(root[-2]=='f') or (root[-2]=='m'):
            print(os.path.join(root,"   ;   ", f),n)
            #print(f)
            a,b,c,d,e=extract_feature(os.path.join(root, f))
            #a=extract_feature(main_dir + i + '/' + f)
            tot= []
            for x in a:tot.append(x)
            for x in b:tot.append(x)
            for x in c:tot.append(x)
            for x in d:tot.append(x)
            for x in e:tot.append(x)
            X4[n]=np.asarray(tot)
            n+=1;
            #print(tot)

In [74]:
len(a),len(b),len(c),len(d),len(e),

(40, 12, 128, 7, 6)

In [75]:
X6= pd.DataFrame(X4.T)

In [76]:
type(X6), X6.shape

(pandas.core.frame.DataFrame, (588, 193))

In [56]:
def decompose_emovo():
        #EMODB_PATH = '/kaggle/input/berlin-database-of-emotional-speech-emodb/wav'
        emotion = []
        path = []
        gender = []  
    
        for root, dirs, files in os.walk(main_dir):
            for name in files:
                if(root[-2]=='f') or (root[-2]=='m'):
                    #print(name[0:3])
                    if name[0:3]=='dis':emotion.append(1)
                    elif name[0:3]=='gio':emotion.append(2)
                    elif name[0:3]=='neu':emotion.append(3)
                    elif name[0:3]=='pau':emotion.append(4)
                    elif name[0:3]=='rab':emotion.append(5)
                    elif name[0:3]=='sor':emotion.append(6)
                    elif name[0:3]=='tri':emotion.append(7)
                    else: emotion.append(0)

                    if name[4]=='f':gender.append(1)
                    elif name[4]=='m':gender.append(2)
                    else: gender.append(0)
                    path.append(os.path.join(  name)) #root ,

        emodb_df = pd.DataFrame(emotion, columns=['labels'])
        emodb_df['source'] = 'EMOVO'
        gender_df = pd.DataFrame(gender, columns=['gender'])
        emodb_df = pd.concat([emodb_df, gender_df, pd.DataFrame(path, columns=['path'])], axis=1)
        
        return emodb_df 

In [57]:
df=decompose_emovo()

In [77]:
df

,labels,source,gender,path
0,1,EMOVO,1,dis-f1-b1.wav
1,1,EMOVO,1,dis-f1-b2.wav
2,1,EMOVO,1,dis-f1-b3.wav
3,1,EMOVO,1,dis-f1-d1.wav
4,1,EMOVO,1,dis-f1-d2.wav
...,...,...,...,...
583,7,EMOVO,2,tri-m3-n1.wav
584,7,EMOVO,2,tri-m3-n2.wav
585,7,EMOVO,2,tri-m3-n3.wav
586,7,EMOVO,2,tri-m3-n4.wav


In [71]:
df[df['labels']==0]

,labels,source,gender,path


In [72]:
df[df['gender']==0]

,labels,source,gender,path


In [78]:
X6

,0,1,2,3,4,5,6,7,8,9,...,183,184,185,186,187,188,189,190,191,192
0,-370.303223,58.331867,-7.632109,15.654898,3.273010,0.692397,-8.063744,5.894156,-5.199350,12.546567,...,17.334037,17.539159,17.562779,34.036367,-0.028319,0.053574,-0.043633,-0.009186,0.019449,-0.007346
1,-374.553833,47.796314,1.653561,20.256063,-6.800525,0.783256,-4.250007,7.473405,-15.591282,14.261038,...,16.263369,17.699334,19.162629,34.229551,0.023288,0.050375,-0.021473,0.040849,0.002533,0.025532
2,-386.473145,75.599594,-15.072874,6.823948,-1.835993,-4.165404,-10.107768,7.045069,-17.148949,11.959360,...,18.860587,18.714656,19.843059,34.157690,0.013844,0.000709,-0.032024,-0.007825,0.018322,0.016368
3,-362.502167,89.977379,-15.514508,5.774208,4.361949,12.591103,-0.374380,-0.392911,-4.743288,15.702434,...,15.868680,18.698590,19.053175,32.441757,0.002924,0.020344,-0.057423,0.041032,-0.014291,-0.002126
4,-412.010406,83.760246,-2.188665,18.547068,0.581788,10.956905,3.381726,3.320800,-9.278383,11.129302,...,16.198743,17.339091,18.793145,33.286765,0.001819,0.007893,-0.003650,0.014794,0.000603,0.001663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,-428.723602,127.801125,15.011064,26.158209,-22.814543,10.531158,8.283092,-1.291359,-13.442948,11.084978,...,16.535757,15.881800,18.443654,33.880198,-0.017768,0.023105,0.130201,0.076282,-0.008200,-0.029686
584,-433.270050,98.356888,24.055376,35.478432,-30.932493,11.173256,12.042026,-1.199043,-12.588658,7.034428,...,16.113687,16.514813,18.799787,34.613071,-0.027987,-0.025462,-0.003617,0.090180,-0.020973,0.000550
585,-458.537323,104.085503,15.052121,37.028015,-10.172383,10.071804,2.412206,-3.682174,-7.252892,11.958858,...,14.285399,16.010688,18.472471,34.431134,0.087554,0.064611,0.170886,-0.068005,0.058233,-0.019411
586,-523.336914,88.056267,18.831060,43.098541,-8.197763,15.856596,4.796964,4.109037,-4.488383,12.122023,...,14.503504,15.854289,17.343597,33.439551,0.031808,-0.014211,0.000701,-0.059435,0.022520,0.021469


In [79]:
X7=pd.concat([X6,df['gender'],df['labels']],axis=1)
#X8=pd.concat([X8,pd.DataFrame(gender),pd.DataFrame(emotion)],axis=1)

In [80]:
X7.head()

,0,1,2,3,4,5,6,7,8,9,...,185,186,187,188,189,190,191,192,gender,labels
0,-370.303223,58.331867,-7.632109,15.654898,3.273010,0.692397,-8.063744,5.894156,-5.199350,12.546567,...,17.562779,34.036367,-0.028319,0.053574,-0.043633,-0.009186,0.019449,-0.007346,1,1
1,-374.553833,47.796314,1.653561,20.256063,-6.800525,0.783256,-4.250007,7.473405,-15.591282,14.261038,...,19.162629,34.229551,0.023288,0.050375,-0.021473,0.040849,0.002533,0.025532,1,1
2,-386.473145,75.599594,-15.072874,6.823948,-1.835993,-4.165404,-10.107768,7.045069,-17.148949,11.959360,...,19.843059,34.157690,0.013844,0.000709,-0.032024,-0.007825,0.018322,0.016368,1,1
3,-362.502167,89.977379,-15.514508,5.774208,4.361949,12.591103,-0.374380,-0.392911,-4.743288,15.702434,...,19.053175,32.441757,0.002924,0.020344,-0.057423,0.041032,-0.014291,-0.002126,1,1
4,-412.010406,83.760246,-2.188665,18.547068,0.581788,10.956905,3.381726,3.320800,-9.278383,11.129302,...,18.793145,33.286765,0.001819,0.007893,-0.003650,0.014794,0.000603,0.001663,1,1


In [81]:
X7.tail()

,0,1,2,3,4,5,6,7,8,9,...,185,186,187,188,189,190,191,192,gender,labels
583,-428.723602,127.801125,15.011064,26.158209,-22.814543,10.531158,8.283092,-1.291359,-13.442948,11.084978,...,18.443654,33.880198,-0.017768,0.023105,0.130201,0.076282,-0.008200,-0.029686,2,7
584,-433.270050,98.356888,24.055376,35.478432,-30.932493,11.173256,12.042026,-1.199043,-12.588658,7.034428,...,18.799787,34.613071,-0.027987,-0.025462,-0.003617,0.090180,-0.020973,0.000550,2,7
585,-458.537323,104.085503,15.052121,37.028015,-10.172383,10.071804,2.412206,-3.682174,-7.252892,11.958858,...,18.472471,34.431134,0.087554,0.064611,0.170886,-0.068005,0.058233,-0.019411,2,7
586,-523.336914,88.056267,18.831060,43.098541,-8.197763,15.856596,4.796964,4.109037,-4.488383,12.122023,...,17.343597,33.439551,0.031808,-0.014211,0.000701,-0.059435,0.022520,0.021469,2,7
587,-524.293213,96.265175,25.127802,48.898922,-5.117021,5.594874,2.337670,3.805295,-5.918553,10.875184,...,18.290698,33.796141,0.006743,-0.004643,-0.009114,-0.008600,0.008819,0.005309,2,7


In [82]:
X7.to_csv("featureEMOVO.csv",index=False,header=True)
#audio_df.to_csv("audioELM.csv",index=False,header=True)

In [83]:
dir ='D:/codes/datasets/ravdes/'
X7.to_excel(dir+"featureEMOVO.xlsx",index=False) 

In [84]:
import os
data = pd.read_csv('featureEMOVO.csv')

In [85]:
data

,0,1,2,3,4,5,6,7,8,9,...,185,186,187,188,189,190,191,192,gender,labels
0,-370.303223,58.331867,-7.632109,15.654898,3.273010,0.692397,-8.063744,5.894156,-5.199350,12.546567,...,17.562779,34.036367,-0.028319,0.053574,-0.043633,-0.009186,0.019449,-0.007346,1,1
1,-374.553833,47.796314,1.653561,20.256063,-6.800525,0.783256,-4.250007,7.473405,-15.591282,14.261038,...,19.162629,34.229551,0.023288,0.050375,-0.021473,0.040849,0.002533,0.025532,1,1
2,-386.473145,75.599594,-15.072874,6.823948,-1.835993,-4.165404,-10.107768,7.045069,-17.148949,11.959360,...,19.843059,34.157690,0.013844,0.000709,-0.032024,-0.007825,0.018322,0.016368,1,1
3,-362.502167,89.977379,-15.514508,5.774208,4.361949,12.591103,-0.374380,-0.392911,-4.743288,15.702434,...,19.053175,32.441757,0.002924,0.020344,-0.057423,0.041032,-0.014291,-0.002126,1,1
4,-412.010406,83.760246,-2.188665,18.547068,0.581788,10.956905,3.381726,3.320800,-9.278383,11.129302,...,18.793145,33.286765,0.001819,0.007893,-0.003650,0.014794,0.000603,0.001663,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,-428.723602,127.801125,15.011064,26.158209,-22.814543,10.531158,8.283092,-1.291359,-13.442948,11.084978,...,18.443654,33.880198,-0.017768,0.023105,0.130201,0.076282,-0.008200,-0.029686,2,7
584,-433.270050,98.356888,24.055376,35.478432,-30.932493,11.173256,12.042026,-1.199043,-12.588658,7.034428,...,18.799787,34.613071,-0.027987,-0.025462,-0.003617,0.090180,-0.020973,0.000550,2,7
585,-458.537323,104.085503,15.052121,37.028015,-10.172383,10.071804,2.412206,-3.682174,-7.252892,11.958858,...,18.472471,34.431134,0.087554,0.064611,0.170886,-0.068005,0.058233,-0.019411,2,7
586,-523.336914,88.056267,18.831060,43.098541,-8.197763,15.856596,4.796964,4.109037,-4.488383,12.122023,...,17.343597,33.439551,0.031808,-0.014211,0.000701,-0.059435,0.022520,0.021469,2,7


In [86]:
data.shape

(588, 195)